In [3]:
import os
import locale
from enum import Enum

import pandas as pd 
import numpy as np 

from matplotlib import pyplot as plt 
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec


locale.setlocale(locale.LC_ALL, 'it_IT.UTF-8')
#locale.setlocale(locale.LC_ALL, '')


'it_IT.UTF-8'

In [4]:
# Daily deaths toll.
daily_data_file_path = os.path.join("..","data", "Istat", "dati-giornalieri-comune")
daily_data_file_name = "comune_giorno.csv"
daily_file = os.path.join(daily_data_file_path, daily_data_file_name)

# Aggregate table.
aggregate_tbl_file_path = os.path.join("..","data", "Istat", "tavola-sintetica-16aprile")
aggregate_tbl_file_name = "Tavola sintetica.xlsx"
aggregate_file = os.path.join(aggregate_tbl_file_path, aggregate_tbl_file_name)



In [13]:
#
# Data file reading.
#
dtype_dict = {
    "PROV":"object"
   ,"REG":"object"
   ,"GE":"object"
   ,"CL_ETA":"object"
}

daily_df = pd.read_csv(daily_file, sep=',', lineterminator='\n',low_memory=False, encoding='latin-1',dtype=dtype_dict)

daily_df.set_index(["NOME_PROVINCIA","NOME_COMUNE"], inplace = True)
daily_df.sort_index(inplace=True)

daily_df.rename(columns={"TOTALE_20\r": "TOTALE_20"}, inplace=True)

# Unpack the MONTHDAY column in two separate colums.
daily_df["Month"] = daily_df["GE"].apply(lambda row : row[0:2])
daily_df["Day"] = daily_df["GE"].apply(lambda row : row[-2:])

(daily_df.shape, daily_df.dtypes)

((849120, 27),
 REG                 object
 PROV                object
 NOME_REGIONE        object
 COD_PROVCOM          int64
 DATA_INIZIO_DIFF    object
 CL_ETA              object
 GE                  object
 MASCHI_15            int64
 MASCHI_16            int64
 MASCHI_17            int64
 MASCHI_18            int64
 MASCHI_19            int64
 MASCHI_20            int64
 FEMMINE_15           int64
 FEMMINE_16           int64
 FEMMINE_17           int64
 FEMMINE_18           int64
 FEMMINE_19           int64
 FEMMINE_20           int64
 TOTALE_15            int64
 TOTALE_16            int64
 TOTALE_17            int64
 TOTALE_18            int64
 TOTALE_19            int64
 TOTALE_20            int64
 Month               object
 Day                 object
 dtype: object)

In [10]:
daily_df.head(3)

REG PROV NOME_REGIONE  COD_PROVCOM  \
NOME_PROVINCIA NOME_COMUNE                                      
Agrigento      Agrigento    19  084      Sicilia        84001   
               Agrigento    19  084      Sicilia        84001   
               Agrigento    19  084      Sicilia        84001   

                           DATA_INIZIO_DIFF CL_ETA    GE  MASCHI_15  \
NOME_PROVINCIA NOME_COMUNE                                            
Agrigento      Agrigento     Dati 2020 n.d.     13  0101          0   
               Agrigento     Dati 2020 n.d.     14  0101          0   
               Agrigento     Dati 2020 n.d.     15  0101          0   

                            MASCHI_16  MASCHI_17  ...  FEMMINE_19  FEMMINE_20  \
NOME_PROVINCIA NOME_COMUNE                        ...                           
Agrigento      Agrigento            0          0  ...           0        9999   
               Agrigento            1          0  ...           0        9999   
               Agrigento            0          0  ...           0        9999   

                            TOTALE_15  TOTALE_16  TOTALE_17  TOTALE_18  \
NOME_PROVINCIA NOME_COMUNE                                               
Agrigento      Agrigento            1          0          0          0   
               Agrigento            0          2          0          1   
               Agrigento            0          0          0          0   

                            TOTALE_19  TOTALE_20\r  Month  Day  
NOME_PROVINCIA NOME_COMUNE                                      
Agrigento      Agrigento            0         9999     01   01  
               Agrigento            0         9999     01   01  
               Agrigento            0         9999     01   01  

[3 rows x 27 columns]

In [14]:
# ------------------------------------------------------------
# ------------------------------------------------------------
# Some common functions.
# ------------------------------------------------------------
# ------------------------------------------------------------
def remove_tick_lines(which, ax):
    if which == 'y':
        for line in ax.yaxis.get_majorticklines():
            line.set_visible(False)
    elif which == 'x':
        for line in ax.xaxis.get_majorticklines():
            line.set_visible(False)
    else:
        assert False, "Wrong axis parameter."

def every_nth_tick(ax, every_nth = 2):
    for n, label in enumerate(ax.xaxis.get_ticklabels()):
        if n % every_nth != 0:
            label.set_visible(False)

def autolabel(rects, ax, dec_no=0):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width() / 2., height + (height * 0.01),
                '%s' % round(height, dec_no),
                ha='center', va='bottom')

def set_axes_common_properties(axe, no_grid=False, border=False):
    rv = 1
    try:
        axe.spines['top'].set_visible(border)
        axe.spines['left'].set_visible(border)
        axe.spines['right'].set_visible(border)
        axe.spines['bottom'].set_visible(border)
        if no_grid == False:
            axe.grid(color='#636262', linestyle='-.', linewidth=0.2)
        rv = 0
    except Exception as ex:
        print("Errore - {e}".format(e=str(ex)))
        
    return rv

def text_box(ax, text, colors=["#FFFFFF", "#000000"], fontsize=14, x=0, y=0):
    """
    """
    rv = False
    try:
        edgecolor = "none"
        boxstyle = "square"
        if len(colors) >= 3 and colors[2] is not None:
            edgecolor = colors[2]
            boxstyle = "round,pad=1"
        ax.text(x, y
               ,text
               ,ha="left", va="center" 
               ,bbox=dict(boxstyle = boxstyle, facecolor = colors[0], edgecolor = edgecolor)
               ,color=colors[1]
               ,fontsize=fontsize)
        set_axes_common_properties(ax, no_grid=True)
        ax.get_xaxis().set_ticks([])
        ax.get_yaxis().set_ticks([])
        
    except Exception as ex:
        print("text_box failed - {ex}".format(ex=ex))
    else:
        rv = True    
    return rv   

#----------------------------------------------------------------
# Common chart function.
#----------------------------------------------------------------
def composed_chart(ax, data_sets,log=False, title=None):
    """
    
    :param ax:
    :param data_sts: dictionary containing all the data set 
                     parameters to customize a single plot;
    :param log:
    :param title:
    :return:
    """
    rv = False
    try:
        set_axes_common_properties(ax, no_grid=False)
        #ax.set_xlim(time_limits)

        max_y = None
        for key in data_sets.keys():
            df = data_sets[key]["df"]
            x = df["timeStamp"].dt.to_pydatetime()
            
            if data_sets[key].get("value column") is not None:
                col_name = data_sets[key]["value column"]
                y = df[col_name]
            else:
                y = df["Totale"]
            new_max = y.max()
            if max_y is None or new_max > max_y:
                max_y = new_max
                
            colors = data_sets[key]["colors"]

            time_limits = [x.min() - pd.Timedelta(hours=10) 
                          ,x.max() + pd.Timedelta(hours=10)]
            ax.set_xlim(time_limits)
            
            ax.scatter(x, y, color=colors[0], s=data_sets[key]["size"], marker=data_sets[key]["dot-shape"], label=data_sets[key]["label"])
            if data_sets[key]["no_plot"] == False:
                ax.plot(x, y, 'b-', linewidth=data_sets[key]["line-size"], color=colors[1])
        
        if title is not None:
            ax.set_title(title, fontsize=16)
        
        ax.set_ylim(top=max_y + 1000)
        
        ax.set_ylabel("Numero", fontsize=12)
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
       
        ax.tick_params(axis='x', labelrotation=80)
        remove_tick_lines('x', ax)
        
        # 'upper left' - 'upper right' - 'lower right'
        ax.legend(fontsize=12, loc='upper left')
            
    except Exception as ex:
        print("composed_c_virus_chart failed - {ex}".format(ex=ex))
    else:
        rv = True    
    return rv

In [12]:
daily_df.loc[("Milano", "Segrate")]

REG PROV NOME_REGIONE  COD_PROVCOM  \
NOME_PROVINCIA NOME_COMUNE                                      
Milano         Segrate      03  015    Lombardia        15205   
               Segrate      03  015    Lombardia        15205   
               Segrate      03  015    Lombardia        15205   
               Segrate      03  015    Lombardia        15205   
               Segrate      03  015    Lombardia        15205   
...                         ..  ...          ...          ...   
               Segrate      03  015    Lombardia        15205   
               Segrate      03  015    Lombardia        15205   
               Segrate      03  015    Lombardia        15205   
               Segrate      03  015    Lombardia        15205   
               Segrate      03  015    Lombardia        15205   

                           DATA_INIZIO_DIFF CL_ETA    GE  MASCHI_15  \
NOME_PROVINCIA NOME_COMUNE                                            
Milano         Segrate             8 aprile      6  0101          0   
               Segrate             8 aprile     16  0101          0   
               Segrate             8 aprile     18  0101          0   
               Segrate             8 aprile     14  0102          0   
               Segrate             8 aprile     17  0102          0   
...                                     ...    ...   ...        ...   
               Segrate             8 aprile     12  0429          0   
               Segrate             8 aprile     12  0430          0   
               Segrate             8 aprile     16  0430          0   
               Segrate             8 aprile     17  0430          1   
               Segrate             8 aprile     18  0430          0   

                            MASCHI_16  MASCHI_17  ...  FEMMINE_19  FEMMINE_20  \
NOME_PROVINCIA NOME_COMUNE                        ...                           
Milano         Segrate              0          0  ...           0           0   
               Segrate              0          1  ...           0           0   
               Segrate              0          0  ...           0           0   
               Segrate              0          0  ...           0           1   
               Segrate              1          1  ...           0           0   
...                               ...        ...  ...         ...         ...   
               Segrate              0          0  ...           0        9999   
               Segrate              0          0  ...           0        9999   
               Segrate              0          0  ...           0        9999   
               Segrate              0          0  ...           0        9999   
               Segrate              0          0  ...           0        9999   

                            TOTALE_15  TOTALE_16  TOTALE_17  TOTALE_18  \
NOME_PROVINCIA NOME_COMUNE                                               
Milano         Segrate              0          0          0          1   
               Segrate              0          0          1          1   
               Segrate              0          0          0          1   
               Segrate              0          0          0          0   
               Segrate              0          1          1          0   
...                               ...        ...        ...        ...   
               Segrate              1          0          0          0   
               Segrate              1          0          0          0   
               Segrate              0          0          0          0   
               Segrate              1          1          0          1   
               Segrate              0          0          0          1   

                            TOTALE_19  TOTALE_20  Month  Day  
NOME_PROVINCIA NOME_COMUNE                                    
Milano         Segrate              0          0     01   01  
               Segrate              0          0   